Primeiro, vamos realizar a importação de todas as bibliotecas que usaremos.

In [337]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.model_selection import KFold, train_test_split, RandomizedSearchCV, GridSearchCV, cross_val_predict, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, make_scorer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier, GradientBoostingClassifier, ExtraTreesClassifier, BaggingClassifier, AdaBoostClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, Lasso, RidgeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from scipy.stats import uniform, randint, loguniform
import xgboost as xgb
import lightgbm as lgb
import joblib
import warnings

Agora, vamos importar nossos dados.

In [249]:
df_test = pd.read_csv('../test.csv')
df = pd.read_csv('../train.csv')

In [250]:
pd.set_option('display.max_columns', None) # Configuracao para mostrar todas as colunas

## Entendendo o significado de todas as colunas

É de extrema importância entender todas as colunas, por isso, vamos copiar e colar o significado delas:

- track_id: O ID único de cada música
    
- artists: Nome dos(as) artistas que performaram a música, separados por ';'
    
- album_name: Nome do álbum no qual aparece a música
    
- track_name: Nome da música
    
- duration_ms: A duração da música em milissegundos
    
- explicit: Boolean indicando se a música possui conteúdo explícito
    
- danceability: Descreve quanto uma música é "dançante" (0.0 = menos dançante, 1.0 = mais dançante)
    
- energy: Representa a intensidade e atividade de uma música (0.0 = baixa energia, 1.0 = alta energia)
    
- key: A tonalidade musical da faixa mapeada usando a notação padrão de Classe de Altura (12 notas musicais)
    
- loudness: Nível geral de volume da faixa em decibéis (dB)
    
- mode: Indica a modalidade (maior ou menor) da faixa
    
- speechiness: Detecta a presença de palavras faladas na faixa
    
- acousticness: Medida de confiança sobre se a faixa é acústica (0,0 = não acústica, 1,0 = altamente acústica)
    
- instrumentalness: Prediz se uma faixa contém vocais (0,0 = contém vocais, 1,0 = instrumental)
    
- liveness: Detecta a presença de uma audiência na gravação (0,0 = gravação em estúdio, 1,0 = performance ao vivo)
    
- valence: Mede a positividade musical transmitida por uma faixa (0,0 = negativa, 1,0 = positiva)
    
- tempo: Tempo estimado da faixa em batidas por minuto (BPM)
    
- time_signature: Assinatura de tempo estimada da faixa (de 3 a 7)
    
- track_genre: O gênero da música
    
- popularity_target: Boolean indicando se a música é popular ou não

Realizando a análise das colunas, fiquei em dúvida sobre a diferença entre a coluna 'tempo' e 'time_signature'. Por isso, realizei uma análise mais profunda no significado delas:

* tempo (Batidas por minuto):
   * O tempo representa a velocidade ou ritmo de uma faixa, medido em batidas por minuto (BPM).
   * O tempo médio é de cerca de 122 BPM, o que representa um ritmo moderado e enérgico.
   * O tempo mínimo é 0 BPM (o que pode indicar dados ausentes ou faixas muito lentas), e o máximo é cerca de 223 BPM (muito rápido).

* time_signature (Fórmula de compasso):
   * A fórmula de compasso representa a estrutura rítmica de uma faixa, indicando quantas batidas existem em cada compasso e qual valor de nota representa uma batida.
   * Os valores geralmente variam de 3 a 7, sendo 4 o mais comum (representando o compasso 4/4, que é padrão em muitos gêneros).
   * A partir das estatísticas, podemos ver que a mediana e os percentis 25 e 75 são 4, confirmando que o compasso 4/4 é realmente o mais comum.
   * O mínimo de 0 pode indicar dados ausentes ou fórmulas de compasso não convencionais.

   Primeiro dou uma olhada por cima dos dados apenas para entender quais colunas são númericas e categóricas. Também tenho a intenção de ver elas como dados reais e por isso chamo o comando `df.head(3)`

In [ ]:
df.head(3)

In [ ]:
df.info()

Agora partindo para uma análise mais profunda das colunas numéricas, começo com uma análise da estatística descritiva de cada coluna.

In [ ]:
df.describe()

Com o output acima, já é possível observar que valores iguais a 0 em 'tempo' e 'time_signature' provavelmente são missing values. Chegamos a essa conclusão porque músicas com 0 BPM não existem, mesmo muito lentas. E 'time_sigture' já comenta na definição das colunas que os valores vão de 3 a 7, ou seja, 0 é incorreto. Não realizamos a correção desses valores faltantes nesse exato momento, mas em breve voltaremos com eles.

## Exploração e Visualização dos Dados

Agora, na intenção de entender os dados e descobrir padrões, vamos realizar uma exploração e visualização dos dados! Para isso, vamos utilizar bibliotecas como Matplot e Seaborn. Nossa missão aqui é descobrir padrões, correlações e tendências nos dados. Vamos usar visualizações eficazes para comunicar os insights e justificar nossas futuras escolhas de features e modelos.

In [254]:
# Definindo um estilo para nossos graficos e alterando o tamanho padrão das figuras geradas pelo Matplotlib
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

Para o começo da nossa análise, é importante saber a quantidade de músicas que são populares ou não e saber como está a distribuição entre elas. No gráfico gerado a seguir, é possível observar que existe uma divisão bem harmoniosa entre músicas populares e não populares.

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='popularity_target', data=df)
plt.title('Distribution of Song Popularity')
plt.xlabel('Popularity (0: Not Popular, 1: Popular)')
plt.ylabel('Count')
plt.show()

Partindo com nossa análise, vamos agora observar nossas variáveis numéricas e a relação entre elas. O Heatmap é um gráfico perfeito para isso que demonstra as relações entre as colunas. Resumindo bastante funciona da forma a seguir: 

Os valores de correlação variam de -1 a 1:

- **+1**: Correlação positiva perfeita. À medida que uma variável aumenta, a outra também aumenta.
- **-1**: Correlação negativa perfeita. À medida que uma variável aumenta, a outra diminui.
- **0**: Nenhuma correlação. As duas variáveis não afetam uma à outra.

In [ ]:
numerical_features = ['duration_ms', 'danceability', 'energy', 'key', 'loudness', 'speechiness', 
                      'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
plt.figure(figsize=(12, 10))
sns.heatmap(df[numerical_features].corr(), annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Heatmap of Numerical Features')
plt.show()

Realizando a análise do Heatmap acima, é possível identificar alguns padrões importantes sobre nossa base de dados.

1. Correlação forte entre 'energy' e 'loudness': Existe uma correlação positiva forte (0.76) entre 'energy' e 'loudness', indicando que músicas mais enérgicas tendem a ser mais altas.

2. Correlação negativa entre 'acousticness' e 'energy': Há uma forte correlação negativa (-0.73) entre 'acousticness' e 'energy', sugerindo que músicas mais acústicas tendem a ser menos enérgicas.

3. Correlação negativa entre 'acousticness' e 'loudness': Similarmente, existe uma correlação negativa moderada (-0.59) entre 'acousticness' e 'loudness', indicando que músicas acústicas tendem a ser menos altas.

4. Correlação positiva entre 'danceability' e 'valence': Há uma correlação positiva moderada (0.48) entre 'danceability' e 'valence', sugerindo que músicas mais dançantes tendem a ter um tom emocional mais positivo.

5. Correlação negativa entre 'instrumentalness' e 'loudness': Observa-se uma correlação negativa moderada (-0.43) entre 'instrumentalness' e 'loudness', indicando que músicas mais instrumentais tendem a ser menos altas.

6. Pouca correlação com 'duration_ms': A 'duration_ms' tem correlações fracas com a maioria das outras características, sugerindo que o comprimento da música não está fortemente relacionado com suas outras propriedades acústicas.

7. Correlações fracas com 'key': A 'key' tem correlações muito fracas com outras características, indicando que não há uma relação forte entre a tonalidade e outros aspectos musicais neste conjunto de dados.

8. Correlações moderadas com 'valence': A 'valence' tem correlações moderadas positivas com 'danceability' (0.48) e 'energy' (0.26), sugerindo que músicas mais positivas tendem a ser mais dançantes e enérgicas.

9. Pouca correlação entre 'tempo' e outras características: O 'tempo' da música tem correlações relativamente fracas com outras características, com a mais forte sendo com 'energy' (0.24).

10. Correlação fraca entre 'speechiness' e outras características: 'Speechiness' tem correlações geralmente fracas com outras características, com a mais notável sendo com 'liveness' (0.21).

11. 'Liveness' tem correlações fracas: A característica 'liveness' não mostra correlações fortes com nenhuma outra característica, sugerindo que é relativamente independente das outras propriedades musicais.

Saber desses padrão será importante para quando começarmos a selecionar as features para o treinamento do nosso modelo! Porque aqui é possível observar que alguns colunas estão extremamente ligadas com outras.

Agora, queremos entender como diferentes características musicais se relacionam com a popularidade das músicas. Por isso, vamos criar alguns gráficos para realizar essa visualização.

Com essa análise, vamos identificar quais características musicais têm maior influência na popularidade de uma música.

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness']
for i, feature in enumerate(features):
    sns.boxplot(x='popularity_target', y=feature, data=df, ax=axes[i//3][i%3])
    axes[i//3][i%3].set_title(f'{feature.capitalize()} by Popularity')
plt.tight_layout()
plt.show()

Observando o gráfico acima, conseguimos ter alguns insights.

1. Danceability (Dançabilidade):

Há uma ligeira tendência de músicas populares (1) terem maior dançabilidade.
A diferença é pequena, mas notável, o que sugere que músicas mais dançantes têm uma probabilidade um pouco maior de serem populares.

2.Instrumentalness (Instrumentalidade):

Músicas populares tendem a ter menor instrumentalidade.
Isso sugere que músicas com vocais são geralmente mais populares do que músicas puramente instrumentais.

Agora, para entender quais gêneros proporcionam mais sucessos às músicas, vamos criar um gráfico para visualizar os 10 melhores gêneros em termos de músicas populares.

In [ ]:
top_genres = df['track_genre'].value_counts().nlargest(10)
plt.figure(figsize=(12, 6))
sns.barplot(x=top_genres.index, y=top_genres.values)
plt.title('Top 10 Genres by Count')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
sns.scatterplot(data=df, x='energy', y='danceability', hue='popularity_target', palette='viridis')
plt.title('Energy vs. Danceability (Colored by Popularity)')
plt.show()

Análisando o gráfico acima, podemos observar que a popularidade não parece ser exclusivamente determinada por altos níveis de energia e dançabilidade, já que músicas populares existem em diversos níveis de energia e dançabilidade, sugerindo que outros fatores também influenciam a popularidade.

In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(data=df, x='duration_ms', bins=50, kde=True)
plt.title('Distribution of Song Durations')
plt.xlabel('Duration (ms)')
plt.show()

Observando a duração das músicas, podemos observar que há uma concentração muito alta de músicas com duração menor. Também observamos que existem músicas com durações extremamente longas. Estes podem ser álbuns inteiros, performances ao vivo, ou erros nos dados.

- A assimetria da distribuição pode afetar análises estatísticas, sendo necessário considerar transformações ou métodos robustos. Em breve voltaremos nesse tópico ao tratar os dados.

Já o gráfico de barras empilhadas abaixo apresenta uma comparação visual da proporção de conteúdo explícito entre músicas populares e não populares 

In [ ]:
explicit_by_popularity = df.groupby('popularity_target')['explicit'].value_counts(normalize=True).unstack()
explicit_by_popularity.plot(kind='bar', stacked=True)
plt.title('Proportion of Explicit Content by Popularity')
plt.xlabel('Popularity')
plt.ylabel('Proportion')
plt.legend(title='Explicit', loc='center left', bbox_to_anchor=(1, 0.5))
plt.tight_layout()
plt.show()

Observando o gráfico acima, podemos observar que:

1. A maioria das músicas, tanto populares quanto não populares, não contém conteúdo explícito.
2. Há uma ligeira tendência de músicas populares terem uma proporção um pouco maior de conteúdo explícito em comparação com as não populares.
3. A presença de conteúdo explícito não parece ser um fator determinante para a popularidade de uma música, dada a pequena diferença observada.

Agora para fechar nossa análise, vamos comparar a distribuição da duração das músicas entre as categorias de popularidade.

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxenplot(x='popularity_target', y='duration_ms', data=df)
plt.title('Song Duration Distribution by Popularity')
plt.xlabel('Popularity (0: Not Popular, 1: Popular)')
plt.ylabel('Duration (ms)')
plt.show()

Observando o gráfico acima, vemos que:

1. As medianas de duração para músicas populares e não populares são muito semelhantes, sugerindo que a duração por si só não é um forte indicador de popularidade.
2. A dispersão (representada pelo tamanho das caixas) é ligeiramente menor para músicas populares, indicando uma maior consistência na duração dessas faixas.
3. Existem muitos outliers em ambas as categorias, representando músicas com durações excepcionalmente longas.
4. As músicas populares parecem ter menos outliers extremos em comparação com as não populares, especialmente na faixa de duração mais longa.
5. A maioria das músicas, independentemente da popularidade, tem duração dentro de um intervalo relativamente estreito, como evidenciado pelo tamanho das caixas do boxplot.
6. Há uma leve tendência de músicas populares terem durações um pouco mais curtas, observável pela posição ligeiramente inferior da caixa para músicas populares.

Os insights mais importantes que encontramos em nossa análise incluem: a ligeira tendência de músicas populares terem maior energia e dançabilidade; a predominância de conteúdo não explícito em ambas as categorias de popularidade, com uma sutil inclinação para mais conteúdo explícito em músicas populares; e a observação de que a duração das músicas não é um forte indicador de popularidade, embora músicas populares tendam a ter durações mais consistentes. Além disso, notamos correlações significativas entre certas características musicais, como a relação positiva entre energia e volume (loudness), e negativa entre acústica e energia.

Ainda é muito cedo para entrarmos na escolha das features para nosso modelo, mas já é possível observar que não existe uma coluna em específica que determina a popularidade das músicas, com isso, é bastante provável que vamos utilizar todas as colunas para treinar nosso modelo, excluindo apenas as colunas de identificação.

## Formulação de Hipóteses

#### Hipótese 01: Influência da dançabilidade na era do TikTok

Com a popularização do TikTok, onde coreografias virais são frequentes, músicas mais dançantes podem ter maior probabilidade de se tornarem populares. Esta hipótese sugere uma correlação positiva entre a dançabilidade de uma música e sua popularidade.

In [ ]:
plt.subplot(2, 2, 1)
sns.boxplot(x='popularity_target', y='danceability', data=df)
plt.title('Dançabilidade vs. Popularidade')
plt.xlabel('Popularidade (0: Não Popular, 1: Popular)')
plt.ylabel('Dançabilidade')

É possível observar que a média de músicas dançantes é um pouco mais alto para músicas populares, mas a diferença não é tão grande assim, logo, não indica extrema importância se a música é dançante ou não para ela ser popular.

#### Hipótese 02: Conteúdo explícito em músicas de temática triste

Músicas categorizadas como "sad" ou emocionalmente intensas podem ter uma maior tendência a conter conteúdo explícito.

In [ ]:
plt.subplot(2, 2, 2)
sad_genres = ['sad', 'melancholic', 'emotional']  # Ajuste conforme necessário
df['is_sad'] = df['track_genre'].isin(sad_genres)
sns.barplot(x='is_sad', y='explicit', data=df)
plt.title('Conteúdo Explícito em Músicas Tristes vs. Outras')
plt.xlabel('Gênero Triste')
plt.ylabel('Proporção de Conteúdo Explícito')
df = df.drop(columns='is_sad')

É possível observar que o conteúdo explícito está extremamente presente em músicas classificadas como 'sad', 'melancholic' ou 'emotional'. Comprovando nossa hipótese.

#### Hipótese 03: Popularidade de músicas com temática melancólica

Músicas classificadas como 'sad' estão entre as mais populares, principalmente porque hoje a depressão é o problema do século.

In [ ]:
top_genres = df['track_genre'].value_counts().nlargest(10)
plt.figure(figsize=(12, 6))
sns.barplot(x=top_genres.index, y=top_genres.values)
plt.title('Top 10 Genres by Count')
plt.xticks(rotation=45, ha='right')
plt.show()

É possível ver que o gênero 'sad' é o segundo gênero com maior sucesso de músicas populares. Comprovando nossa hipótese.

#### Hipótese 04: Relação inversa entre instrumentalidade e popularidade

Músicas com alto grau de instrumentalidade (pouco ou nenhum vocal) podem tender a ser menos populares, possivelmente devido à preferência do público mainstream por músicas com letras e vozes.

In [ ]:
plt.subplot(2, 2, 4)
sns.boxplot(x='popularity_target', y='instrumentalness', data=df)
plt.title('Instrumentalidade vs. Popularidade')
plt.xlabel('Popularidade (0: Não Popular, 1: Popular)')
plt.ylabel('Instrumentalidade')

É possível observar que existem músicas populares e que são instrumentais, mas, em maioria, elas são classificadas como não populares. Comprovando nossa hipótese.

## Limpeza e Tratamento de Valores Nulos

### Tratamento de valores duplicados

Fazer a limpeza de valores duplicados é importantes pois valores duplicados podem prejudicar no aprendizado do nosso algoritmo. Para realizar a limpeza, vamos primeiro checar se existem valores duplicados e caso existirem, vamos excluir todos.

In [ ]:
duplicated_values = df[df.duplicated()]
print(len(duplicated_values))

Como podemos ver acima, não existem valores duplicados. Assim, seguiremos com nossa análise

### Tratamento de missing values

Missing values são valores = null

É importante verificar a existência deles e resolver os valores faltantes caso existam.

In [ ]:
df.isnull().sum()

Como podemos ver acima, também não existem valores faltantes.

Agora, vamos verificar valores que são iguais a 0. Como identificamos lá no começo desse notebook, já sabemos de alguns valores que não deveriam estar iguais a 0. Também vamos procurar por novos valores iguais a 0 e que não deveriam estar assim, caso existam, vamos resolver.

In [ ]:
zero_count = (df == 0).sum()
print(zero_count)

Como podemos ver acima, as colunas 'time_signature' e 'tempo' possuem valores iguais a 0 e elas não deveriam estar assim:

1. Tempo:

- Representa a velocidade da música em batidas por minuto (BPM).
- Um valor 0 significaria que não há batidas, o que é musicalmente impossível.
- Tempos típicos variam de cerca de 60 BPM (lento) a 200 BPM (muito rápido).


2. Time signature (fórmula de compasso):

- Indica quantas batidas há por compasso e qual nota representa uma batida.
- É escrita como uma fração, por exemplo, 4/4 ou 3/4.
- Um valor 0 não faria sentido, pois implicaria em nenhuma batida por compasso.

In [ ]:
df['time_signature'].value_counts()

In [ ]:
print(df['tempo'].describe(), df['tempo'].value_counts())

Para resolver esses 2 problemas, vamos: 

- Substituir os valores de 'time_signature' pela moda, que é 4
- Substituir os valores de 'tempo' pela mediana

In [272]:
tempo_median = df['tempo'].median()
df_treating_data = df.copy()
df_treating_data['tempo'] = df_treating_data['tempo'].replace(0,tempo_median)
df_treating_data['time_signature'] = df_treating_data['time_signature'].replace(0,4)

### Identificação de outliers e correção

As colunas que vamos visualizar e depois tratar os outliers são as colunas: 'duration_ms', 'loudness' e 'tempo'.

Realizei a escolha dessas três colunas porque todas as outras colunas numéricas se constituem de valores que vâo de 0 a 1. Algumas até possuem outliers mas são features importantes e que quero manter para preservar a pureza dos dados em features que vão de 0 a 1. Mais tarde, vamos minimizar os impactos que esses outliers podem causar com o standardScaler

Agora partindo para a identificação dos outliers que escolhemos, vamos começar visualizando em boxplot os possíveis outliers.

In [ ]:
sns.boxplot(data=df_treating_data, y='duration_ms')
plt.show()

Observando o gráfico acima, é possívei ver que 'duration_ms' possui vários outliers, vamos guardar essa informação e em breve corrigir eles.

Agora fazendo a análise dos outliers de 'loudness', vamos desenhar outro boxplot para realizar essa visualização:

In [ ]:
sns.boxplot(data=df_treating_data, y='loudness')
plt.show()

Agora fazendo a análise dos outliers de 'tempo', vamos desenhar outro boxplot para realizar essa visualização:

In [ ]:
sns.boxplot(data=df_treating_data, y='tempo')
plt.show()

Após uma análise cuidadosa dos dados, incluindo a presença de outliers em colunas como 'duration_ms', 'loudness' e 'tempo', decidimos manter o dataset em seu estado original, sem realizar nenhuma transformação ou exclusão neste momento.

Esta decisão se baseia nas seguintes considerações:

1. Preservação da integridade dos dados: Cada característica de uma música, mesmo aquelas que parecem estatisticamente atípicas, pode conter informações valiosas sobre a natureza única da faixa e potencialmente sobre sua popularidade.

2. Flexibilidade para análises futuras: Manter todos os dados em seu estado original nos proporciona máxima flexibilidade para realizar diversos tipos de análises e experimentos posteriormente.

3. Seleção de features posterior: Decidimos adiar a seleção de features para uma etapa posterior do processo. Isso nos permitirá tomar decisões mais informadas sobre quais características são mais relevantes para nosso modelo, baseando-nos em análises mais aprofundadas e experimentos iniciais.

4. Captura de padrões complexos: No domínio musical, o que pode parecer um outlier ou uma característica irrelevante pode, na verdade, ser parte de um padrão mais complexo que contribui para a popularidade de uma faixa.

5. Abordagem holística: Ao manter todos os dados, estamos adotando uma visão holística do problema, permitindo que nosso modelo potencialmente descubra relações e padrões que poderíamos não antecipar inicialmente.

6. Desafio realista para o modelo: Trabalhar com dados não processados cria um cenário mais próximo do mundo real, desafiando nosso modelo a lidar com a complexidade e variabilidade inerentes aos dados musicais.

Ao adiar a seleção de features, estamos nos dando a oportunidade de explorar diversas abordagens de modelagem e técnicas de seleção de características. Isso pode incluir métodos como:

- Análise de correlação entre features e a variável alvo (popularidade)
- Técnicas de seleção de features baseadas em importância (como as fornecidas por modelos de árvore)
- Métodos de redução de dimensionalidade (como PCA ou t-SNE)
- Testes de diferentes combinações de features em modelos preliminares

Esta abordagem nos permite ser mais metódicos e baseados em evidências em nossas decisões sobre quais características incluir em nosso modelo final. Também nos dá a flexibilidade de adaptar nossa estratégia conforme ganhamos mais insights sobre os dados e o problema em questão.

Nosso próximo passo será iniciar uma análise exploratória mais aprofundada dos dados, buscando entender melhor as relações entre as diferentes características e a popularidade das músicas. Isso nos ajudará a informar nossas decisões futuras sobre seleção de features e escolha de modelos.

## Codificação de Variáveis Categóricas

In [277]:
def normalize_and_encode_dataset(df, target, n_splits=5):
    encoded_df = df.copy()
    encoding_dict = {}
    scaler_dict = {}

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    for column in df.columns:
        if column == target:
            continue
        
        encoded = np.zeros(len(df))
        column_dict = {}
        
        # Target encoding
        for train_idx, val_idx in kf.split(df):
            target_means = df.iloc[train_idx].groupby(column)[target].mean()
            encoded[val_idx] = df[column].iloc[val_idx].map(target_means)
            
            column_dict.update(target_means.to_dict())
        
        global_mean = df[target].mean()
        encoded = np.where(np.isnan(encoded), global_mean, encoded)
        
        column_dict['_global_mean'] = global_mean
        encoding_dict[column] = column_dict
        
        # Normalization
        scaler = StandardScaler()
        normalized = scaler.fit_transform(encoded.reshape(-1, 1)).flatten()
        
        encoded_df[f"{column}_encoded_normalized"] = normalized
        scaler_dict[column] = scaler
    
    return encoded_df, encoding_dict, scaler_dict

def apply_normalize_and_encode(df, encoding_dict, scaler_dict):
    encoded_df = df.copy()
    
    for column, column_dict in encoding_dict.items():
        if column in df.columns:
            global_mean = column_dict['_global_mean']
            
            # Apply target encoding
            encoded = df[column].map(column_dict)
            encoded = encoded.fillna(global_mean)
            
            # Apply normalization
            scaler = scaler_dict[column]
            normalized = scaler.transform(encoded.values.reshape(-1, 1)).flatten()
            
            encoded_df[f"{column}_encoded_normalized"] = normalized
        else:
            print(f"Warning: Column '{column}' not found in the input DataFrame.")
    
    return encoded_df

In [313]:
new_df = df_treating_data.copy()
new_df = new_df.drop(columns=['track_unique_id', 'track_id'])

In [314]:
encoded_df, encoding_dict, scaler_dict = normalize_and_encode_dataset(new_df, 'popularity_target')

In [315]:
encoded_df = encoded_df.drop(columns=['artists', 'album_name', 'track_name', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'track_genre'])

In [316]:
X = encoded_df.drop('popularity_target', axis=1)
y = encoded_df['popularity_target']

In [ ]:
X['artists_album_track'] = X['artists_encoded_normalized'] * X['album_name_encoded_normalized'] * X['track_name_encoded_normalized']
X = X.drop(columns=['instrumentalness_encoded_normalized', 'duration_ms_encoded_normalized', 'key_encoded_normalized', 'loudness_encoded_normalized', 'mode_encoded_normalized', 'time_signature_encoded_normalized', 'explicit_encoded_normalized'])
X.head()

In [ ]:
X_temp_df_no_genres, X_test_df_no_genres, y_temp_df_no_genres, y_test_df_no_genres = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_df_no_genres, X_val_df_no_genres, y_train_df_no_genres, y_val_df_no_genres = train_test_split(X_temp_df_no_genres, y_temp_df_no_genres, test_size=0.25, random_state=42)

print(f"Tamanho do conjunto de treino: {len(X_train_df_no_genres)}")
print(f"Tamanho do conjunto de validação: {len(X_val_df_no_genres)}")
print(f"Tamanho do conjunto de teste: {len(X_test_df_no_genres)}")

In [284]:
rf_classifier = RandomForestClassifier(random_state=42)

xgb_model = xgb.XGBClassifier(random_state=42)

lgb_model = lgb.LGBMClassifier(random_state=42)

logistic_model = LogisticRegression(random_state=42)

etc_model = ExtraTreesClassifier(random_state=42)

gbc_model = GradientBoostingClassifier(random_state=42)

In [285]:
param_grid_rf = {
    'n_estimators': randint(100, 1000),
    'max_depth': [None] + list(range(5, 31)),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'max_features': ['auto', 'sqrt', 'log2']
}

param_grid_xgb = {
    'n_estimators': randint(100, 1000),
    'max_depth': randint(3, 15),
    'learning_rate': uniform(0.01, 0.3),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4)
}

param_grid_lgb = {
    'n_estimators': randint(100, 1000),
    'max_depth': randint(3, 15),
    'learning_rate': uniform(0.01, 0.3),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4)
}

param_grid_logistic = {
    'C': uniform(0.1, 10),
    'penalty': ['l1', 'l2', 'elasticnet'],
    'solver': ['saga'],
    'l1_ratio': uniform(0, 1)
}

param_grid_etc = {
    'n_estimators': randint(100, 1000),
    'max_depth': [None] + list(range(5, 31)),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'max_features': ['auto', 'sqrt', 'log2']
}

param_grid_gbc = {
    'n_estimators': randint(100, 1000),
    'learning_rate': uniform(0.01, 0.3),
    'max_depth': randint(3, 15),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'subsample': uniform(0.6, 0.4)
}

In [286]:
models = [
    (rf_classifier, param_grid_rf, 'Random Forest'),
    (xgb_model, param_grid_xgb, 'XGBoost'),
    (lgb_model, param_grid_lgb, 'LightGBM'),
    (logistic_model, param_grid_logistic, 'Logistic Regression'),
    (etc_model, param_grid_etc, 'Extra Trees'),
    (gbc_model, param_grid_gbc, 'Gradient Boosting')
]

In [ ]:
accuracy_scorer = make_scorer(accuracy_score)

best_params = {}
best_scores = {}
for model, param_grid, name in models:
    print(f"Tuning {name}...")
    rs = RandomizedSearchCV(model, param_grid, n_iter=100, cv=5, n_jobs=-1, random_state=42, verbose=1, scoring=accuracy_scorer)
    rs.fit(X_train_df_no_genres, y_train_df_no_genres)
    best_params[name] = rs.best_params_
    best_scores[name] = rs.best_score_

for name in best_params.keys():
    print(f"\nBest parameters for {name}:")
    for param, value in best_params[name].items():
        print(f"    {param}: {value}")
    print(f"Best accuracy for {name}: {best_scores[name]:.4f}")

best_model = max(best_scores, key=best_scores.get)
print(f"\nOverall best model: {best_model}")
print(f"Best overall accuracy: {best_scores[best_model]:.4f}")

Best parameters for Random Forest:

1. max_depth: 24
2. max_features: log2
3. min_samples_leaf: 1
4. min_samples_split: 4
5. n_estimators: 227

- Best accuracy for Random Forest: 0.9055

Best parameters for XGBoost:

1. colsample_bytree: 0.679486272613669
2. learning_rate: 0.01165663513708072
3. max_depth: 13
4. n_estimators: 598
5. subsample: 0.8827429375390468

- Best accuracy for XGBoost: 0.9066

Best parameters for LightGBM:

1. colsample_bytree: 0.8732027093665427
2. learning_rate: 0.031356594538068695
3. max_depth: 10
4. n_estimators: 704
5. subsample: 0.9379501243877818

- Best accuracy for LightGBM: 0.9061

Best parameters for Logistic Regression:

1. C: 6.274815096277165
2. l1_ratio: 0.6116531604882809
3. penalty: l1
4. solver: saga

- Best accuracy for Logistic Regression: 0.8971

Best parameters for Extra Trees:

1. max_depth: 24
2. max_features: log2
3. min_samples_leaf: 1
4. min_samples_split: 4
5. n_estimators: 227

- Best accuracy for Extra Trees: 0.9044

Best parameters for Gradient Boosting:
    
1. learning_rate: 0.08323765667433225
2. max_depth: 13
3. min_samples_leaf: 9
4. min_samples_split: 18
5. n_estimators: 929
6. subsample: 0.9887128330883843

- Best accuracy for Gradient Boosting: 0.9078


In [333]:
rf_classifier = RandomForestClassifier(random_state=42, max_depth=24, max_features='log2', min_samples_leaf=1, min_samples_split=4, n_estimators=227)

xgb_model = xgb.XGBClassifier(random_state=42, colsample_bytree=0.679486272613669, learning_rate=0.01165663513708072, max_depth=13, n_estimators=598, subsample=0.8827429375390468)

lgb_model = lgb.LGBMClassifier(random_state=42, colsample_bytree=0.8732027093665427, learning_rate=0.031356594538068695, max_depth=10, n_estimators=704,
subsample=0.9379501243877818)

logistic_model = LogisticRegression(random_state=42, C=6.274815096277165, l1_ratio=0.6116531604882809, penalty='l1', solver='saga')

etc_model = ExtraTreesClassifier(random_state=42, max_depth=24, max_features='log2', min_samples_leaf=1, min_samples_split=4, n_estimators=227)

gbc_model = GradientBoostingClassifier(random_state=42, learning_rate=0.08323765667433225, max_depth=13, min_samples_leaf=9, min_samples_split=18, n_estimators=929, subsample=0.9887128330883843)

svc_model = SVC(random_state=42, C=1, kernel='rbf')

ada_clf = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=10), n_estimators=500, learning_rate=0.01, random_state=42)

hist_gbc = HistGradientBoostingClassifier(max_iter=100, random_state=42, early_stopping=True, l2_regularization=0.1, learning_rate=0.1, max_bins=255, max_depth=20, min_samples_leaf=10)

In [ ]:
voting_model = VotingClassifier(
    estimators=[
        ('lgb_model', lgb_model),
        ('xgb_model', xgb_model),
        ('gbc_model', gbc_model),
    ],
    voting='soft'
)

voting_model.fit(X_train_df_no_genres, y_train_df_no_genres)

y_pred_val = voting_model.predict(X_val_df_no_genres)
y_pred_test = voting_model.predict(X_test_df_no_genres)

print("Validation Accuracy:", accuracy_score(y_val_df_no_genres, y_pred_val))
print("Test Accuracy:", accuracy_score(y_test_df_no_genres, y_pred_test))


In [321]:
df_test_target_encoding = df_test.copy()
df_test_target_encoding = df_test_target_encoding.drop(columns=['track_unique_id', 'track_id'])

df_test_target_encoding['tempo'] = df_test_target_encoding['tempo'].replace(0,tempo_median)
df_test_target_encoding['time_signature'] = df_test_target_encoding['time_signature'].replace(0,4)

df_test_target_encoding = apply_normalize_and_encode(df_test_target_encoding, encoding_dict, scaler_dict)

df_test_target_encoding['artists_album_track'] = df_test_target_encoding['artists_encoded_normalized'] * df_test_target_encoding['album_name_encoded_normalized'] * df_test_target_encoding['track_name_encoded_normalized']

df_test_target_encoding = df_test_target_encoding.drop(columns=['artists', 'album_name', 'track_name', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'track_genre', 'instrumentalness_encoded_normalized', 'duration_ms_encoded_normalized', 'key_encoded_normalized', 'loudness_encoded_normalized', 'mode_encoded_normalized', 'time_signature_encoded_normalized', 'explicit_encoded_normalized'])

predict_target_encoding_stacking = voting_model.predict(df_test_target_encoding)
result_stacking = pd.DataFrame({ 'track_unique_id': df_test['track_unique_id'], 'popularity_target': predict_target_encoding_stacking })
result_stacking.to_csv('result_voting.csv', index=False)